In [1]:
import re
import glob, os, shutil
import gzip
import numpy as np
import pandas as pd

In [2]:
from src import ingestion
from src import matrix
from src import model
from sklearn.metrics import accuracy_score


In [23]:
def get_benign_paths(train_outpath, cat='benign'):
    return [train_outpath+'/'+cat+'/'+ d for d in os.listdir(train_outpath+'/'+cat)]

In [3]:
all_paths = ingestion.get_benign_paths('data', 'education')+ingestion.get_malware_paths('/datasets/dsc180a-wi20-public/Malware/amd_data_smali', 20)




In [4]:
app_smalis = [ingestion.smalis_from_paths(ingestion.get_smali_paths(p)) for p in all_paths]





In [5]:
smalis = pd.DataFrame(app_smalis)

In [6]:
A, B, P = matrix.construct_matrices(app_smalis)

In [19]:
kernel_train = A.dot(B).dot(P).dot(B).dot(A.T).todense()

In [20]:
svc = SVC(kernel='precomputed')
svc.fit(kernel_train, [0]*20+[1]*20)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='precomputed', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [21]:
y_pred = svc.predict(kernel_train)
accuracy_score([0]*20+[1]*20, y_pred)

0.5

In [24]:
B.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [25]:
def construct_kernel(A, B, P):
    return [A.dot(A.T), A.dot(B).dot(A.T), A.dot(P).dot(A.T), A.dot(B).dot(P).dot(B).dot(A.T)]

In [26]:
kernels = construct_kernel(A, B, P)